In [18]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [19]:
import os
from dotenv import load_dotenv
import requests
from IPython.display import Image, display
import re
import textwrap

In [20]:
import sqlite3
from datetime import datetime
import gc
import pickle

In [21]:
load_dotenv()  

TMDB_TOKEN = os.getenv("TMDB_TOKEN")
if not TMDB_TOKEN:
    raise ValueError("TMDB_TOKEN not found in .env")

BASE_URL = "https://api.themoviedb.org/3"
IMAGE_BASE = "https://image.tmdb.org/t/p/w200"  # small poster image
HEADERS = {"Authorization": f"Bearer {TMDB_TOKEN}"}

In [22]:
## Remove year and extra characters from movie title.
## Example: "Toy Story (1995)" -> "Toy Story"
def clean_movie_title(title):
    ## Remove the year in parentheses in the movie csv
    title = re.sub(r'\s*\(\d{4}\)\s*$', '', title)
    title = title.strip()
    return title

In [23]:
## Extract year from title if present.
## Example: "Toy Story (1995)" -> "1995
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    return match.group(1) if match else None

In [24]:
## Search TMDB for a movie title and return overview + poster URL.
## Automatically cleans title and extracts year.
def get_tmdb_info(title):
    ## Extract year and clean title
    year = extract_year(title)
    clean_title = clean_movie_title(title)
    
    ## Search parameters
    params = {
        "query": clean_title,
        "include_adult": False
    }
    
    ## Add year if available (helps narrow results)
    if year:
        params["year"] = year
    
    try:
        response = requests.get(
            f"{BASE_URL}/search/movie",
            headers=HEADERS,
            params=params,
            timeout=10
        )
        response.raise_for_status()
        data = response.json()
        
        ## if movie exists get the overview of the movie and the poster
        if data.get('results'):
            movie = data['results'][0]
            overview = movie.get('overview', 'No overview available')
            poster_path = movie.get('poster_path')
            poster_url = IMAGE_BASE + poster_path if poster_path else None
            
            return overview, poster_url
        else:
            return "No overview found", None
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching TMDB info for '{clean_title}': {e}")
        return "Error fetching overview", None



In [25]:
## Load CSV data into SQLite database
def setup_movie_database():
    conn = sqlite3.connect('movies.db')
    
    print("Loading ratings and movies into database...")
    
    ratings_df = pd.read_csv("100k_ratings.csv")
    movies_df = pd.read_csv("100k_movies.csv")
    
    ratings_df.to_sql('ratings', conn, if_exists='replace', index=False)
    movies_df.to_sql('movies', conn, if_exists='replace', index=False)
    
    conn.execute('CREATE INDEX IF NOT EXISTS idx_user ON ratings(userId)')
    conn.execute('CREATE INDEX IF NOT EXISTS idx_movie ON ratings(movieId)')
    conn.execute('CREATE INDEX IF NOT EXISTS idx_movie_id ON movies(movieId)')
    
    conn.commit()
    print(f"Loaded {len(ratings_df)} ratings and {len(movies_df)} movies")
    conn.close()

In [26]:
## Create database for storing user recommendations
def create_recommendations_db():
    conn = sqlite3.connect('recommendations.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS user_recommendations (
            user_id INTEGER,
            movie_id INTEGER,
            predicted_rating REAL,
            title TEXT,
            overview TEXT,
            poster_url TEXT,
            recommendation_type TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (user_id, movie_id, recommendation_type)
        )
    ''')
    
    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_user_rec 
        ON user_recommendations(user_id, recommendation_type)
    ''')
    
    conn.commit()
    return conn

In [27]:
## Create database for caching TMDB API results
def create_tmdb_cache_db():
    conn = sqlite3.connect('tmdb_cache.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tmdb_cache (
            movie_id INTEGER PRIMARY KEY,
            title TEXT,
            overview TEXT,
            poster_url TEXT,
            cached_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    return conn

In [28]:
setup_movie_database()
create_recommendations_db()
create_tmdb_cache_db()
print("All databases created!")

Loading ratings and movies into database...
Loaded 100836 ratings and 9742 movies
All databases created!


In [29]:
## Load data from database instead of CSV
def load_data_from_db():
    conn = sqlite3.connect('movies.db')
    
    ratings_df = pd.read_sql('SELECT * FROM ratings', conn)
    movies_df = pd.read_sql('SELECT * FROM movies', conn)
    
    conn.close()
    return ratings_df, movies_df

In [30]:
ratings_df, movies_df = load_data_from_db()

## Merge both data frames into one
df = pd.merge(ratings_df, movies_df[['movieId', 'genres']], on='movieId', how='left')

df

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Horror
100834,610,168252,5.0,1493846352,Action|Sci-Fi


In [31]:
## map each unique userid or movieid to an integer
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
## convert multiple genres into binary columns(Drama|Horror-> a 1 per column)
mlb = MultiLabelBinarizer()

## encode userId's since ML models do not work with string Id's
df['userId'] = user_encoder.fit_transform(df['userId'])
df['movieId'] = movie_encoder.fit_transform(df['movieId'])

## update df by splitting the genres into a column with 1 or 0
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres').str.split('|')), columns = mlb.classes_, index = df.index))

In [32]:
df.drop(columns = "(no genres listed)", inplace = True)

In [33]:
train_df, test_df = train_test_split(df, test_size = 0.2)
train_df

,userId,movieId,rating,timestamp,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
49849,317,9471,3.0,1536152813,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
58552,380,7074,4.0,1262898702,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
55234,366,697,4.0,997812240,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
49530,317,6895,3.5,1275845365,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57171,379,1692,3.0,1494803711,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,18,157,2.0,965707724,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
81908,516,8256,3.0,1487953575,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
92863,598,497,0.5,1519144562,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
43127,287,6204,3.0,1182171727,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
reader = Reader(rating_scale = (0.5, 5)) ## ratings go from .5 to 5
## loads datafram into Surpise dataset
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

In [35]:
## creates a matrix factorization recommender
model_svd = SVD()
model_svd.fit(trainset)

## building antiset
predictions_svd = model_svd.test(trainset.build_anti_testset())
accuracy.rmse(predictions_svd)

RMSE: 0.4744


0.47435128452483216

In [36]:
## Save the trained model
with open('model_svd.pkl', 'wb') as f:
    pickle.dump(model_svd, f)
print("Model saved to model_svd.pkl")

## Save the encoders
encoders = {
    'user_encoder': user_encoder,
    'movie_encoder': movie_encoder
}
with open('encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)
print("Encoders saved to encoders.pkl")

Model saved to model_svd.pkl
Encoders saved to encoders.pkl


In [37]:
## Save recommendations to database
def save_recommendations(user_id, recommendations, recommendation_type="top"):
    conn = None
    try:
        conn = create_recommendations_db()
        cursor = conn.cursor()
        
        for rec in recommendations:
            cursor.execute('''
                INSERT OR REPLACE INTO user_recommendations 
                (user_id, movie_id, predicted_rating, title, overview, poster_url, recommendation_type)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (user_id, rec['movieId'], rec['predicted_rating'], 
                  rec['title'], rec['overview'], rec['poster_url'], recommendation_type))
        
        conn.commit()
        print(f"Saved {len(recommendations)} {recommendation_type} recommendations for user {user_id}")
    except Exception as e:
        print(f"Error saving recommendations: {e}")
    finally:
        if conn:
            conn.close()

In [38]:
## Retrieve saved recommendations from database
def get_saved_recommendations(user_id, recommendation_type="top", n=5):
    conn = create_recommendations_db()
    
    query = '''
        SELECT movie_id as movieId, title, predicted_rating, overview, poster_url
        FROM user_recommendations 
        WHERE user_id = ? AND recommendation_type = ?
        ORDER BY predicted_rating DESC 
        LIMIT ?
    '''
    
    cursor = conn.cursor()
    cursor.execute(query, (user_id, recommendation_type, n))
    """[
        {
            "movieId": 12,
            "title": "Toy Story",
            "predicted_rating": 4.6,
            "overview": "...",
            "poster_url": "poster.jpg"
        }
        ]
    """
    columns = [desc[0] for desc in cursor.description]
    results = [dict(zip(columns, row)) for row in cursor.fetchall()]
    
    conn.close()
    return results if results else None

In [39]:
## Get TMDB info with database caching to avoid repeated API calls
def get_tmdb_info_cached(title, movie_id):
    ## Convert movie_id to regular Python int to avoid datatype mismatch
    movie_id = int(movie_id)
    
    conn = None
    try:
        conn = create_tmdb_cache_db()
        cursor = conn.cursor()
        
        ## Check cache first
        cursor.execute('SELECT overview, poster_url FROM tmdb_cache WHERE movie_id = ?', (movie_id,))
        result = cursor.fetchone()
        
        if result:
            return result[0], result[1]
        
        ## If not cached, fetch from TMDB API
        overview, poster_url = get_tmdb_info(title)
        
        ## Save to cache
        cursor.execute('''
            INSERT OR REPLACE INTO tmdb_cache (movie_id, title, overview, poster_url)
            VALUES (?, ?, ?, ?)
        ''', (movie_id, title, overview, poster_url))
        
        conn.commit()
        
        return overview, poster_url
        
    except Exception as e:
        print(f"Database error for movie {movie_id}: {e}")
        ## If database fails, just return from API without caching
        return get_tmdb_info(title)
    finally:
        if conn:
            conn.close()

In [40]:
## Clear all recommendations for a user
def clear_user_recommendations(user_id):
    conn = create_recommendations_db()
    cursor = conn.cursor()
    cursor.execute('DELETE FROM user_recommendations WHERE user_id = ?', (user_id,))
    conn.commit()
    deleted = cursor.rowcount
    conn.close()
    print(f"Cleared {deleted} recommendations for user {user_id}")


In [41]:
## Get statistics about TMDB cache
def get_cache_stats():
    conn = create_tmdb_cache_db()
    cursor = conn.cursor()
    
    cursor.execute('SELECT COUNT(*) FROM tmdb_cache')
    count = cursor.fetchone()[0]
    
    cursor.execute('SELECT MIN(cached_at), MAX(cached_at) FROM tmdb_cache')
    dates = cursor.fetchone()
    
    conn.close()
    
    print(f"TMDB Cache Statistics:")
    print(f"  Cached movies: {count}")
    if dates[0]:
        print(f"  First cached: {dates[0]}")
        print(f"  Last cached: {dates[1]}")

In [42]:
## Get top N movie recommendations with database caching
def get_top_n_recommendations(user_id, model_svd, df, movies_df, movie_encoder, n=5, use_cache=True):
    
    ## Check if we have cached recommendations
    if use_cache:
        cached = get_saved_recommendations(user_id, "top", n)
        if cached:
            print(f"Using cached recommendations for user {user_id}")
            return cached
    
    print(f"Computing new recommendations for user {user_id}...")
    
    ## find movies user has not rated yet
    user_movies = df[df['userId'] == user_id]['movieId'].unique()
    all_movies = df['movieId'].unique()
    movies_to_predict = list(set(all_movies) - set(user_movies))

    ## build pairs for prediction
    user_movie_pairs = [(user_id, movie_id, 0) for movie_id in movies_to_predict]
    predictions_cf = model_svd.test(user_movie_pairs)

    top_n_recommendations = sorted(predictions_cf, key=lambda x: x.est, reverse=True)[:n]

    results = []
    ## decode movie ID and get the results
    for pred in top_n_recommendations:
        movie_id_encoded = int(pred.iid)
        predicted_rating = pred.est
        
        ## convert back to the MovieLens ID
        movie_id_original = movie_encoder.inverse_transform([movie_id_encoded])[0]
        
        ## find the movies metadata
        movie_row = movies_df[movies_df['movieId'] == movie_id_original]
        if movie_row.empty:
            continue
            
        title = movie_row['title'].values[0]
        
        ## Use cached TMDB data
        overview, poster_url = get_tmdb_info_cached(title, movie_id_original)
        
        results.append({
            "movieId": movie_id_original,
            "title": title,
            "predicted_rating": predicted_rating,
            "overview": overview,
            "poster_url": poster_url
        })
    
    ## Save to recommendations database
    save_recommendations(user_id, results, "top")
    
    return results

In [43]:
## Get bottom N movie recommendations with database caching
def get_bottom_n_recommendations(user_id, model_svd, df, movies_df, movie_encoder, n=5, use_cache=True):
    
    if use_cache:
        cached = get_saved_recommendations(user_id, "bottom", n)
        if cached:
            print(f"Using cached bottom recommendations for user {user_id}")
            return cached
    
    print(f"Computing new bottom recommendations for user {user_id}...")
    
    user_movies = df[df['userId'] == user_id]['movieId'].unique()
    all_movies = df['movieId'].unique()
    movies_to_predict = list(set(all_movies) - set(user_movies))

    user_movie_pairs = [(user_id, movie_id, 0) for movie_id in movies_to_predict]
    predictions_cf = model_svd.test(user_movie_pairs)

    bottom_n_recommendations = sorted(predictions_cf, key=lambda x: x.est)[:n]

    results = []
    for pred in bottom_n_recommendations:
        movie_id_encoded = int(pred.iid)
        predicted_rating = pred.est
        
        movie_id_original = movie_encoder.inverse_transform([movie_id_encoded])[0]
        
        movie_row = movies_df[movies_df['movieId'] == movie_id_original]
        if movie_row.empty:
            continue
            
        title = movie_row['title'].values[0]
        
        overview, poster_url = get_tmdb_info_cached(title, movie_id_original)
        
        results.append({
            "movieId": movie_id_original,
            "title": title,
            "predicted_rating": predicted_rating,
            "overview": overview,
            "poster_url": poster_url
        })
    ## bottom instead of top to get the movies to not watch
    save_recommendations(user_id, results, "bottom")
    
    return results


In [44]:
## Display recommendations in a nice format with posters
def display_recommendations(user_id, recommendations, recommendation_type="Top"):
    print(f"\n{'='*80}")
    print(f"{recommendation_type} {len(recommendations)} Movie Recommendations for User {user_id}")
    print(f"{'='*80}\n")
    
    for idx, movie in enumerate(recommendations, 1):
        print(f"{idx}. {movie['title']}")
        print(f"   Predicted Rating: {movie['predicted_rating']:.2f}/5.0")
        
        ## Wrap overview text to 75 characters per line
        wrapped_overview = textwrap.fill(movie['overview'], width=75, 
                                         initial_indent="   Overview: ",
                                         subsequent_indent="             ")
        print(wrapped_overview)
        
        ## if the movie has a poster display it. if not print not available
        if movie['poster_url']:
            print(f"   Poster URL: {movie['poster_url']}")
            try:
                display(Image(url=movie['poster_url'], width=200))
            except Exception as e:
                print(f"   Could not display poster: {e}")
        else:
            print("   (No poster available)")
        
        print()

Code to fix a locked database error

In [45]:
## Force close all connections
gc.collect()

## Recreate the databases with proper cleanup
create_tmdb_cache_db()
create_recommendations_db()
print("Databases ready!")

Databases ready!


In display_recommendations, if you want the top 5 recommended movies as the 3rd paramter put "Top"
If you want the "do not watch" movies type "bottom"

In [46]:
user_id = 13
top_movies = get_top_n_recommendations(user_id, model_svd, df, movies_df, movie_encoder, n=5)
display_recommendations(user_id, top_movies, "Top")

Computing new recommendations for user 13...
Saved 5 top recommendations for user 13

Top 5 Movie Recommendations for User 13

1. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
   Predicted Rating: 4.40/5.0
   Overview: After the insane General Jack D. Ripper initiates a nuclear
             strike on the Soviet Union, a war room full of politicians,
             generals and a Russian diplomat all frantically try to stop
             it.
   Poster URL: https://image.tmdb.org/t/p/w200/6x7MzQ6BOMlRzam1StcmPO9v61g.jpg



2. Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
   Predicted Rating: 4.40/5.0
   Overview: At a tiny Parisian café, the adorable yet painfully shy Amélie
             accidentally discovers a gift for helping others. Soon Amelie
             is spending her days as a matchmaker, guardian angel, and all-
             around do-gooder. But when she bumps into a handsome stranger,
             will she find the courage to become the star of her very own
             love story?
   Poster URL: https://image.tmdb.org/t/p/w200/vZ9NhNbQQ3yhtiC5sbhpy5KTXns.jpg



3. Dark Knight, The (2008)
   Predicted Rating: 4.34/5.0
   Overview: Batman raises the stakes in his war on crime. With the help of
             Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets
             out to dismantle the remaining criminal organizations that
             plague the streets. The partnership proves to be effective,
             but they soon find themselves prey to a reign of chaos
             unleashed by a rising criminal mastermind known to the
             terrified citizens of Gotham as the Joker.
   Poster URL: https://image.tmdb.org/t/p/w200/qJ2tW6WMUDux911r6m7haRef0WH.jpg



4. Boot, Das (Boat, The) (1981)
   Predicted Rating: 4.33/5.0
   Overview: A German submarine hunts allied ships during the Second World
             War, but it soon becomes the hunted. The crew tries to survive
             below the surface, while stretching both the boat and
             themselves to their limits.
   Poster URL: https://image.tmdb.org/t/p/w200/u8FhQPncOAkwcei2OI9orPWhV6K.jpg



5. Rear Window (1954)
   Predicted Rating: 4.30/5.0
   Overview: A wheelchair-bound photographer spies on his neighbors from
             his apartment window and becomes convinced one of them has
             committed murder.
   Poster URL: https://image.tmdb.org/t/p/w200/ILVF0eJxHMddjxeQhswFtpMtqx.jpg


In [47]:
user_id = 15
bottom_movies = get_bottom_n_recommendations(user_id, model_svd, df, movies_df, movie_encoder, n=5)
display_recommendations(user_id, bottom_movies, "Bottom (Do Not Watch)")

Computing new bottom recommendations for user 15...
Saved 5 bottom recommendations for user 15

Bottom (Do Not Watch) 5 Movie Recommendations for User 15

1. Wild Wild West (1999)
   Predicted Rating: 2.04/5.0
   Overview: Legless Southern inventor Dr. Arliss Loveless plans to
             rekindle the Civil War by assassinating President Ulysses S.
             Grant. Only two men can stop him: gunfighter James West and
             master-of-disguise and inventor Artemus Gordon. The two must
             team up to thwart Loveless' plans.
   Poster URL: https://image.tmdb.org/t/p/w200/mCdo7nykEVCa25bjnkwgyX35fjm.jpg



2. Godzilla (1998)
   Predicted Rating: 2.04/5.0
   Overview: French nuclear tests irradiate an iguana into a giant monster
             that viciously attacks freighter ships in the Pacific Ocean. A
             team of experts, including Niko Tatopoulos, conclude that the
             oversized reptile is the culprit. Before long, the giant
             lizard is loose in Manhattan as the US military races to
             destroy the monster before it reproduces and it's spawn takes
             over the world.
   Poster URL: https://image.tmdb.org/t/p/w200/xJVl1I95StraYAwaNbBkVoWE2qA.jpg



3. Jason X (2002)
   Predicted Rating: 2.08/5.0
   Overview: In the year 2455, Old Earth is now a contaminated planet
             abandoned for centuries -- a brown world of violent storms,
             toxic landmasses and poisonous seas. Yet humans have returned
             to the deadly place that they once fled, not to live, but to
             research the ancient, rusting artifacts of the long-gone
             civilizations. But it's not the harmful environment that could
             prove fatal to the intrepid, young explorers who have just
             landed on Old Earth. For them, it's Friday the 13th, and Jason
             lives!
   Poster URL: https://image.tmdb.org/t/p/w200/ggOND0hfoE0f3K857joSzEeIchb.jpg



4. Stuart Saves His Family (1995)
   Predicted Rating: 2.10/5.0
   Overview: A self-help advocate struggles to put his dysfunctional family
             in its place.
   Poster URL: https://image.tmdb.org/t/p/w200/80YaWpUokDoPynsJi0ittpts3J0.jpg



5. Speed 2: Cruise Control (1997)
   Predicted Rating: 2.11/5.0
   Overview: A disgruntled former employee hijacks the Seabourn Legend
             cruise liner. Set on a fixed course, without any means of
             communication and at the mercy of the hijacker, it's up to the
             one cop on vacation, and his soon to be fiancé (hopefully)
             Annie, to regain control of it before it kills the passengers
             and causes an environmental disaster. Insurmountable and
             daunting tasks await them on their  perilous journey
             throughout the ship trying to fend off the hijacker and save
             the passengers.
   Poster URL: https://image.tmdb.org/t/p/w200/gnK1ocpwUTj24zAktzomOJsD2bu.jpg
